In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, Input, Flatten, Conv2D, AveragePooling2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.model_selection import train_test_split

In [ ]:
x_train = np.load('/content/drive/MyDrive/IJCNN/x_train.npy')
x_test = np.load('/content/drive/MyDrive/IJCNN/x_test.npy')
y_train = np.load('/content/drive/MyDrive/IJCNN/y_train.npy')
y_test = np.load('/content/drive/MyDrive/IJCNN/y_test.npy')
print(y_train.sum()/y_train.shape[0], y_test.sum()/y_test.shape[0])

0.5005160989578131 0.4984517031265608


In [ ]:
model = Sequential((
    Input((12210,)),
    Dense(4096, activation='relu'),
    Dense(12210, activation='sigmoid')
))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              50016256  
_________________________________________________________________
dense_1 (Dense)              (None, 12210)             50024370  
Total params: 100,040,626
Trainable params: 100,040,626
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=SGD(0.01), loss=tf.keras.losses.MSE)
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
939/939 [==============================] - 16s 15ms/step - loss: 0.2595 - val_loss: 0.2519
Epoch 2/10
939/939 [==============================] - 14s 15ms/step - loss: 0.2513 - val_loss: 0.2504
Epoch 3/10
939/939 [==============================] - 14s 14ms/step - loss: 0.2503 - val_loss: 0.2501
Epoch 4/10
939/939 [==============================] - 14s 15ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 5/10
939/939 [==============================] - 14s 15ms/step - loss: 0.2499 - val_loss: 0.2499
Epoch 6/10
939/939 [==============================] - 14s 15ms/step - loss: 0.2499 - val_loss: 0.2499
Epoch 7/10
939/939 [==============================] - 14s 14ms/step - loss: 0.2499 - val_loss: 0.2499
Epoch 8/10
939/939 [==============================] - 14s 14ms/step - loss: 0.2499 - val_loss: 0.2498
Epoch 9/10
939/939 [==============================] - 14s 15ms/step - loss: 0.2498 - val_loss: 0.2498
Epoch 10/10
939/939 [==============================] - 14s 14ms/step - loss: 0.249

In [ ]:
extractor = Model(inputs=model.input, outputs=model.get_layer('dense').output)
extractor.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12210)]           0         
_________________________________________________________________
dense (Dense)                (None, 4096)              50016256  
Total params: 50,016,256
Trainable params: 50,016,256
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train2 = extractor.predict(x_train)
x_test2 = extractor.predict(x_test)

In [ ]:
model = Sequential((
    Input((4096,)),
    Dense(1024),
    Activation('relu'),
    Dense(1, activation='sigmoid')
))

In [ ]:
model.compile(optimizer=SGD(0.01), loss=BinaryCrossentropy(), metrics=['accuracy'])
model.fit(x_train2, y_train, batch_size=32, epochs=10, validation_data=(x_test2, y_test))

Epoch 1/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6678 - accuracy: 0.6026 - val_loss: 0.6481 - val_accuracy: 0.6212
Epoch 2/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6436 - accuracy: 0.6292 - val_loss: 0.6405 - val_accuracy: 0.6281
Epoch 3/10
939/939 [==============================] - 3s 4ms/step - loss: 0.6341 - accuracy: 0.6413 - val_loss: 0.6348 - val_accuracy: 0.6344
Epoch 4/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6280 - accuracy: 0.6492 - val_loss: 0.6305 - val_accuracy: 0.6372
Epoch 5/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6291 - accuracy: 0.6453 - val_loss: 0.6272 - val_accuracy: 0.6391
Epoch 6/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6215 - accuracy: 0.6515 - val_loss: 0.6239 - val_accuracy: 0.6420
Epoch 7/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6173 - accuracy: 0.6559 - val_loss: 0.6208 - val_accuracy: 0.6450
Epoch 

In [ ]:
model.evaluate(x_train2, y_train)
model.evaluate(x_test2, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.6149 - accuracy: 0.6566


[0.6149353981018066, 0.6565777659416199]

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n, activation='relu'):
        super().__init__()
        #self.batch_norm_1 = BatchNormalization()
        self.activation_1 = Activation(activation)
        self.dense_1 = Dense(n)
        #self.batch_norm_2 = BatchNormalization()
        self.activation_2 = Activation(activation)
        self.dense_2 = Dense(n)
    
    def call(self, x, training=False):
        #x2 = self.batch_norm_1(x, training=training)
        x2 = self.activation_1(x)#x2)
        x2 = self.dense_1(x2, training=training)
        
        #x2 = self.batch_norm_2(x2, training=training)
        x2 = self.activation_2(x2)
        x2 = self.dense_2(x2, training=training)
        
        return x + x2

In [ ]:
model = Sequential((
    Input((4096,)),
    Dense(1024),
    ResidualBlock(1024, activation='relu'),
    Activation('relu'),
    Dense(1, activation='sigmoid')
))

In [ ]:
model.compile(optimizer=SGD(0.01), loss=BinaryCrossentropy(), metrics=['accuracy'])
model.fit(x_train2, y_train, batch_size=32, epochs=10, validation_data=(x_test2, y_test))

Epoch 1/10
939/939 [==============================] - 5s 4ms/step - loss: 0.6652 - accuracy: 0.6074 - val_loss: 0.6440 - val_accuracy: 0.6226
Epoch 2/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6363 - accuracy: 0.6372 - val_loss: 0.6339 - val_accuracy: 0.6330
Epoch 3/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6297 - accuracy: 0.6387 - val_loss: 0.6254 - val_accuracy: 0.6425
Epoch 4/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6174 - accuracy: 0.6579 - val_loss: 0.6221 - val_accuracy: 0.6491
Epoch 5/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6092 - accuracy: 0.6671 - val_loss: 0.6134 - val_accuracy: 0.6574
Epoch 6/10
939/939 [==============================] - 4s 4ms/step - loss: 0.6025 - accuracy: 0.6718 - val_loss: 0.6077 - val_accuracy: 0.6614
Epoch 7/10
939/939 [==============================] - 4s 4ms/step - loss: 0.5963 - accuracy: 0.6734 - val_loss: 0.6008 - val_accuracy: 0.6655
Epoch 

In [ ]:
model.evaluate(x_train2, y_train)
model.evaluate(x_test2, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.5853 - accuracy: 0.6870


[0.5852788090705872, 0.6870442628860474]